In [1]:
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install --no-index ../input/detectron2-offline/detectron2/detectron2-0.6-cp37-cp37m-linux_x86_64.whl --find-links=../input/detectron2-offline/detectron2

Processing /kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2
  Preparing metadata (setup.py) ... - \ done
  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python3.7 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2/setup.py'"'"'; __file__='"'"'/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-knf9_amv
       cwd: /kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2/
  Complete output (5 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  error: could not create 'build': Read-only file system
  --

In [2]:
import detectron2
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
import os

from PIL import Image
import numpy as np  
import pandas as pd 
import cv2  
import matplotlib.pyplot as plt

In [3]:
cfg = get_cfg()

cfg.INPUT.MASK_FORMAT='bitmask'
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_cells_train",)
cfg.DATASETS.TEST = ("sartorius_cells_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001 
cfg.SOLVER.MAX_ITER = 3000    
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .35
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.7


In [4]:
ids, masks=[],[]
test_names = os.listdir('../input/sartorius-cell-instance-segmentation/test')

cfg = get_cfg()
from detectron2.projects.point_rend import add_pointrend_config
add_pointrend_config(cfg)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
cfg.MODEL.WEIGHTS = '../input/trained-model/model_final_3_5it_lr1.pth'  
cfg.TEST.DETECTIONS_PER_IMAGE = 1500
predictor = DefaultPredictor(cfg)
THRESHOLDS = [.15, .35, .55] 
MIN_PIXELS = [75, 150, 75]

In [5]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
def get_masks(fn, predictor):
    im = cv2.imread(str(fn))
    pred = predictor(im)
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= THRESHOLDS[pred_class]
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    res = []
    used = np.zeros(im.shape[:2], dtype=int) 
    for mask in pred_masks:
        mask = mask * (1-used)
        if mask.sum() >= MIN_PIXELS[pred_class]: # skip predictions with small area
            used += mask
            res.append(rle_encode(mask))
    return res

In [6]:
for fn in test_names:
    print(fn[:-4])
    file = os.path.join("../input/sartorius-cell-instance-segmentation/test", fn)
    encoded_masks = get_masks(file, predictor)
    for enc in encoded_masks:
        ids.append(fn[:-4])
        masks.append(enc)
        
df = pd.DataFrame({'id':ids, 'predicted':masks})
df.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

7ae19de7bc2a
d8bfd1dafdc4
d48ec7815252


,id,predicted
0,7ae19de7bc2a,178898 3 179601 4 180304 5 181008 6 181711 7 1...
1,7ae19de7bc2a,318197 10 318898 14 319599 17 320301 19 321002...
2,7ae19de7bc2a,140002 16 140706 22 141410 22 142114 23 142819...
3,7ae19de7bc2a,146004 3 146708 8 147412 10 148116 12 148820 1...
4,7ae19de7bc2a,60101 5 60804 7 61507 9 62210 10 62913 12 6361...
